In [3]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch import nn
import numpy as np
from torch.utils.data import Dataset, TensorDataset, DataLoader

In [4]:
data = pd.read_csv("C:\\Users\\garya\\ML projects\\Stock analysis\\modelling\\dataset\\deeplearningdataset.csv")

In [5]:
X_data, y_data = data[['margin', "ebitda", 'revenue', "profit"]], data[['Change']]

In [6]:
X_data = X_data.to_numpy()

In [7]:
y_data = y_data.to_numpy()
temp = []
for i in y_data:
    if(i > 0):
        temp.append(1)
    else:
        temp.append(0)
y_data = np.array(temp)

In [8]:
X_tensor = torch.Tensor(X_data)

In [9]:
y_tensor = torch.Tensor(y_data).type(torch.long)

In [10]:
X_train, X_test, y_train , y_test = train_test_split(X_tensor, y_tensor, test_size = 0.2, random_state=42)

In [11]:
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [12]:
for i, (inputs, labels) in enumerate(train_loader): 
    print(inputs.shape)
    print(labels.shape)
    break

torch.Size([64, 4])
torch.Size([64])


In [18]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(4, 16, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2, padding=1)
        self.fc1 = nn.Linear(32 , 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)
        self.dropout = nn.Dropout(p=0.2)
        self.sigmoid = nn.Sigmoid()
        self.Relu1 = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = x.view(-1, 32)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.Relu1(x)
        x = self.fc2(x)
        x = self.dropout(x)
        x = self.Relu1(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x


In [72]:
epochs = 1000
learning_rate = 0.1
hidden_layers = 64
momentum = 0.9
num_layers = 2
input_size = 4

In [73]:
model_0 = CNN()

In [74]:
# loss_fn = nn.L1Loss()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_0.parameters(), lr = learning_rate , momentum=momentum)

# CNN 

In [75]:
for epoch in range(epochs):
    for i, (inputs, labels) in enumerate(train_loader): 
        #model train mode
        model_0.train()

        # Forward pass
        y_preds = model_0(inputs.unsqueeze(dim=2))
        #calculate the loss 
        
        loss = loss_fn(y_preds, labels)

        #optimizer zero grad
        optimizer.zero_grad()

        #back propagation 
        loss.backward()

        #gradient descent 
        optimizer.step()

    if(epoch%10 == 0):
        model_0.eval()
        with torch.inference_mode():
            y_test_pred = model_0(X_test.unsqueeze(dim=2))
            loss_test = loss_fn(y_test_pred,y_test)

        print(f"Train Loss : {loss.item()} , Test Loss : {loss_test.item()}")

    

Train Loss : 0.6943356990814209 , Test Loss : 0.6926684975624084
Train Loss : 0.7166134715080261 , Test Loss : 0.6901255249977112
Train Loss : 0.6678937673568726 , Test Loss : 0.6921658515930176
Train Loss : 0.6925491690635681 , Test Loss : 0.6924847364425659
Train Loss : 0.6865905523300171 , Test Loss : 0.6882375478744507
Train Loss : 0.6877681612968445 , Test Loss : 0.6865368485450745
Train Loss : 0.6633457541465759 , Test Loss : 0.6876329779624939
Train Loss : 0.6960287690162659 , Test Loss : 0.6929053664207458
Train Loss : 0.6969648003578186 , Test Loss : 0.6896867752075195
Train Loss : 0.6931434273719788 , Test Loss : 0.6930708885192871
Train Loss : 0.6782352328300476 , Test Loss : 0.6891242861747742
Train Loss : 0.680582582950592 , Test Loss : 0.6894286870956421
Train Loss : 0.6859748959541321 , Test Loss : 0.690395712852478
Train Loss : 0.6835548281669617 , Test Loss : 0.6919400691986084
Train Loss : 0.6611807942390442 , Test Loss : 0.6903764605522156
Train Loss : 0.694481790065

 # RNN

In [76]:
class RNNModel(nn.Module):
    def __init__(self):
        super(RNNModel, self).__init__() 
        self.rnn = nn.RNN(4, 256)
        self.l1 = nn.Linear(in_features = 256, out_features = 128)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(in_features = 128 , out_features = 2)
        
    def forward(self, x):
        x,y = self.rnn(x)
#         print(y.shape)
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        return x

In [77]:
model_1 = RNNModel()
epochs = 100
learning_rate = 0.01
hidden_layers = 64
momentum = 0.9
num_layers = 2
input_size = 4
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_1.parameters(), lr = learning_rate , momentum=momentum)

In [78]:
for epoch in range(epochs):
    for i, (inputs, labels) in enumerate(train_loader): 
        #model train mode
        model_0.train()

        # Forward pass
        y_preds = model_1(inputs)
        #calculate the loss 
#         print(y_preds.shape)
#         break
        loss = loss_fn(y_preds, labels)

        #optimizer zero grad
        optimizer.zero_grad()

        #back propagation 
        loss.backward()

        #gradient descent 
        optimizer.step()
#     break
    if(epoch%10 == 0):
        model_0.eval()
        with torch.inference_mode():
            y_test_pred = model_1(X_test)
            loss_test = loss_fn(y_test_pred,y_test)

        print(f"Train Loss : {loss.item()} , Test Loss : {loss_test.item()}")

Train Loss : 0.7044835686683655 , Test Loss : 0.6919167041778564
Train Loss : 0.6837254762649536 , Test Loss : 0.6929894089698792
Train Loss : 0.7006697654724121 , Test Loss : 0.6928457021713257
Train Loss : 0.675035834312439 , Test Loss : 0.6942077875137329
Train Loss : 0.7082319855690002 , Test Loss : 0.6922135353088379
Train Loss : 0.6997717618942261 , Test Loss : 0.6928080916404724
Train Loss : 0.6889241933822632 , Test Loss : 0.6937897801399231
Train Loss : 0.6721938252449036 , Test Loss : 0.6941248178482056
Train Loss : 0.6872169971466064 , Test Loss : 0.6912552118301392
Train Loss : 0.689087450504303 , Test Loss : 0.6948478817939758
